In [2]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import clear_output
import camelot
import re
import warnings
import voila
warnings.filterwarnings("ignore")

# Pulling Indigenous Information from CER Final Reports

Description of app: This app allows users to load a PDF they would like to extract tables from. The app has been specifically designed to focus on tables containing indigenous information in the CER final reports. It can still work for other types of tables, but it isn't guarenteed it will work properly.

## Caveats

The app was created with the consideration that all CER final reports are the same. If the structure changes, it may not extract the tables as well.

## Uploading a PDF of the CER Final Report

Please upload the PDF you are currently interested in. Afterwards, you will be asked to fill in some information and then the table you are looking at. You can only extract one table at a time. Give the page numbers of the table and it will be extracted.

In [3]:
btn_upload = widgets.FileUpload(accept='.pdf', multiple=False)
display(btn_upload)

FileUpload(value={}, accept='.pdf', description='Upload')

In [4]:
output_pages = widgets.Output()
output_section = widgets.Output()
output_appendix = widgets.Output()

output_extract_table = widgets.Output()
output_multi_point_col = widgets.Output()
output_separate_bullets = widgets.Output()
output_save_table = widgets.Output()

In [19]:
# layout = widgets.Layout(width='500px', height='40px') #set width and height
layout = widgets.Layout(display='flex',flex_flow='column')
button_layout = widgets.Layout(width='auto', height='40px') #set width and height
style = {'description_width': 'initial'}

def input_pages():
    global pages_str
    global pages_iter
    pages_widget = widgets.Text(
        value='',
        placeholder='ex: 131, 132, 133, 134',
        description='Pages (comma separated): ',
        disabled=False,
        layout=layout,
        style=style
    )
    display(pages_widget)

    def pages_callback(wdgt):
        global pages_str
        global pages_iter
        
        pages_str = str(wdgt.value)
        display(pages_str)
        pages_iter = [int(item) for item in pages_str.split(',')]
        
        with output_pages:
            clear_output()
            print('Pages: ', pages_str)

    pages_widget.on_submit(pages_callback)
    
def input_section():
    global section_name
    section_widget = widgets.Text(
        value='',
        placeholder='ex: 7.3 Environmental Issues Raised by Participants',
        description='Section Name: ',
        disabled=False,
        layout=layout,
        style=style
    )
    display(section_widget)

    def section_callback(wdgt):
        global section_name
        section_name = str(wdgt.value)
        display(section_name)
        
        with output_section:
            clear_output()
            print('Section Name: ', section_name)

    section_widget.on_submit(section_callback)
    
def input_appendix():
    global appendix_name
    appendix_widget = widgets.Text(
        value='',
        placeholder='ex: Appendix II - Aboriginal Groups that Participated in the OH-02-2015 Proceeding and the Form of their Participation',
        description='Appendix Name: ',
        disabled=False,
        layout=layout,
        style=style
    )
    display(appendix_widget)

    def appendix_callback(wdgt):
        global appendix_name
        appendix_name = str(wdgt.value)
        display(appendix_name)
        
        with output_appendix:
            clear_output()
            print('Appendix Name: ', appendix_name)

    appendix_widget.on_submit(appendix_callback)

In [6]:
def extract_table(b):
    global df
    global pages_iter
    global name_of_document
    with output_extract_table:
        clear_output()
        print('start extracting')        

    [uploaded_file] = btn_upload.value
    file_path = uploaded_file
    
    with output_extract_table:
        print('file path:', file_path)

#     for name, file_info in btn_upload.value.items():
#         with open (name, 'wb') as file:
#             file.write(file_info['content'])

    name_pattern = '([^//]+)$'
    name_of_document = re.findall(name_pattern, file_path)

    with output_extract_table:
        print('pages: ', pages_str)
    
    ## Pulling tables with Camelot

    tables = camelot.read_pdf(uploaded_file, copy_text=['v'], strip_text = '\n', line_scale=40, f = 'excel', flavour = 'lattice', pages=str(pages_str))
#     print(tables[0].df)
#     print(tables[1].df)

    with output_extract_table:
        print('tables: ', len(tables))

    for i, page in enumerate(pages_iter):
#         print(page)
        tables[i].df.rename(columns=tables[i].df.iloc[0], inplace = True)
        tables[i].df.drop([0], inplace = True)
        tables[i].df['page_number'] = page
        tables[i].df['name_of_document'] = name_of_document[0][9:-18]
        # NEB_-_Report_-_Enbridge_-_Line_3_Replacement_Detailed_Assessment
        tables[i].df['date_of_document'] = file_path[-8:-4]
        tables[i].df['name_of_section'] = section_name
        tables[i].df['name_of_appendix'] = appendix_name
        
    if len(tables) > 1:
        df = tables[0].df.append([tables[x].df for x in range(1, len(tables))]).reset_index(drop=True)
    else:
        df = tables[0].df
        
    with output_extract_table:
        print('Finished Extraction')

In [7]:
def input_multi_point_col():
    global multi_point_col_name
    multi_point_col_widget = widgets.Text(
        value='',
        placeholder='ex: Environmental Issue(s) Raised',
        #What is the exact column title for the column with multiple information points?
        description='Name of column with multiple bullet points: ',
        disabled=False,
        layout=layout,
        style=style
    )
    display(multi_point_col_widget)

    def multi_point_col_callback(wdgt):
        global multi_point_col_name
        multi_point_col_name = str(wdgt.value)
        display(multi_point_col_name)
        
        with output_multi_point_col:
            clear_output()
            print('Column Name: ', multi_point_col_name)

    multi_point_col_widget.on_submit(multi_point_col_callback)
    

In [16]:
def separate_bullets(b):
    global df
    with output_separate_bullets:
        clear_output()
        print('Separating column with multiple points per row.')
        
    s = df[multi_point_col_name].str.split('  ').apply(pd.Series, 1).stack()
    s.index = s.index.droplevel(-1)
    s.name = multi_point_col_name

    del df[multi_point_col_name]
    df = df.join(s)

    df[multi_point_col_name] = df[multi_point_col_name].str.replace('', '')
    df[multi_point_col_name] = df[multi_point_col_name].str.replace('  ', '')
    df.drop(df.columns[[0]], axis=1, inplace=True)
    
    with output_separate_bullets:
        clear_output()
        print('Finished separating points.')

In [9]:
def save_table_to_excel(b):
    df.to_excel(name_of_document[0][9:-18] + '_' + str(pages_iter[0]) + '_' + str(pages_iter[-1]) + '.xlsx', index=False)
    
    with output_save_table:
        clear_output()
        print('Saved Table')

In [10]:
input_pages()
display(output_pages)
input_section()
display(output_section)
input_appendix()
display(output_appendix)

Text(value='', description='Pages (comma separated): ', layout=Layout(display='flex', flex_flow='column'), pla…

Output()

Text(value='', description='Section Name: ', layout=Layout(display='flex', flex_flow='column'), placeholder='e…

Output()

Text(value='', description='Appendix Name: ', layout=Layout(display='flex', flex_flow='column'), placeholder='…

Output()

'131, 132, 133, 134'

'7.3 Environmental Issues Raised by Participants'

''

In [11]:
extract_button = widgets.Button(description="Extract Table")
display(extract_button)
display(output_extract_table)

extract_button.on_click(extract_table)

Button(description='Extract Table', style=ButtonStyle())

Output()

In [13]:
input_multi_point_col()
display(output_multi_point_col)

Text(value='', description='Name of column with multiple bullet points: ', layout=Layout(display='flex', flex_…

Output()

'Environmental Issue(s) Raised'

In [21]:
separate_bullets_button = widgets.Button(description="Create new row for each point",
                                         disabled=False,
                                         display='flex',
                                         flex_flow='column',
                                         align_items='stretch', 
                                         layout = button_layout)
display(separate_bullets_button)
display(output_separate_bullets)
separate_bullets_button.on_click(separate_bullets)

Button(description='Create new row for each point', layout=Layout(height='40px', width='auto'), style=ButtonSt…

Output(outputs=({'output_type': 'stream', 'text': 'Separating column with multiple points\n', 'name': 'stdout'…

In [20]:
save_button = widgets.Button(description="Save Table",
                             disabled=False,
                             display='flex',
                             flex_flow='column',
                             align_items='stretch', 
                             layout = button_layout)
display(save_button)
display(output_save_table)
save_button.on_click(save_table_to_excel)

Button(description='Save Table', layout=Layout(height='40px', width='auto'), style=ButtonStyle())

Output(outputs=({'output_type': 'stream', 'text': 'Saved Table\n', 'name': 'stdout'},))

In [ ]:
# # A button example
# from IPython.display import display
# button = widgets.Button(description="Click Me!")
# output = widgets.Output()

# display(button, output)

# def on_button_clicked(b):
#     with output:
#         print("Button clicked.")

# button.on_click(on_button_clicked)